In [1]:
import torch
from sac_FPGA import sac 


In [2]:
ac_kwargs = dict(hidden_sizes=[64,80], activation=torch.nn.ReLU)

logger_kwargs = dict(output_dir='real', exp_name='lambda')


In [ ]:
sac(alpha=0.0001, lr=1e-4, ep_len=80, gradient_steps=80, ac_kwargs=ac_kwargs, logger_kwargs=logger_kwargs)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: julii_tm (julii). Use `wandb login --relogin` to force relogin


Logging data to real\progress.txt
Saving config:

{
    "ac_kwargs":	{
        "activation":	"ReLU",
        "hidden_sizes":	[
            64,
            80
        ]
    },
    "actor_critic":	"MLPActorCritic",
    "alpha":	0.0001,
    "batch_size":	100,
    "device":	"cuda:0",
    "ep_len":	80,
    "episodes_per_epoch":	10,
    "epochs":	100,
    "exp_name":	"lambda",
    "gamma":	0.99,
    "gradient_steps":	80,
    "logger":	{
        "<spinup.utils.logx.EpochLogger object at 0x0000023D2D478890>":	{
            "epoch_dict":	{},
            "exp_name":	"lambda",
            "first_row":	true,
            "log_current_row":	{},
            "log_headers":	[],
            "output_dir":	"real",
            "output_file":	{
                "<_io.TextIOWrapper name='real\\\\progress.txt' mode='w' encoding='cp1252'>":	{
                    "mode":	"w"
                }
            }
        }
    },
    "logger_kwargs":	{
        "exp_name":	"lambda",
        "output_dir":	"real"
    },
 

C:\Users\giuli\Anaconda3\envs\brevitas_env\Lib\site-packages\torch\_tensor.py:1413: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at C:\cb\pytorch_1000000000000\work\c10/core/TensorImpl.h:1928.)
  return super().rename(names)


Sent Scale: 3.980493506420407e-13
Done streaming weights for episode 0.
Ready for episode: 0, test_flag: True.
Received data from episode: 0, test_flag: True.
Sent Scale: 3.980493506420407e-13
Done streaming weights for episode 1.
Ready for episode: 1, test_flag: False.
Received data from episode: 1, test_flag: False.
Sent Scale: 4.552346997629964e-13
Done streaming weights for episode 2.
Ready for episode: 2, test_flag: False.
Received data from episode: 2, test_flag: False.
Sent Scale: 5.100153697805421e-13
Done streaming weights for episode 3.
Ready for episode: 3, test_flag: False.
Received data from episode: 3, test_flag: False.
Sent Scale: 5.619330955414226e-13
Done streaming weights for episode 4.
Ready for episode: 4, test_flag: False.
Received data from episode: 4, test_flag: False.
Sent Scale: 6.830401587214208e-13
Done streaming weights for episode 5.
Ready for episode: 5, test_flag: False.
Received data from episode: 5, test_flag: False.
Sent Scale: 8.329042750637727e-13
Do

In [ ]:
dir_path = logger_kwargs['output_dir']

In [ ]:
server_ip = '192.168.1.10'
server_port = 7

In [ ]:
import socket
client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client.settimeout(60 * 60)  # Set the timeout for blocking socket operations, 1h
client.connect((server_ip, server_port))

In [ ]:
def calculate_reward(o2):
    return o2[0]

from utils import create_input_tensor
import core_FPGA as core

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Have to run inference once - create random data
o = create_input_tensor(core.N_INPUT, device)

# Convert o to tensors and move to the appropriate device
o = torch.as_tensor(o, dtype=torch.float32, device=device).view(1, -1)

In [ ]:
from utils import weights_creation
import struct
import core_FPGA as core
def FPGA_update(o, ep, client):
   
    ac.pi.eval()

    out_brevitas = ac.pi.twin(o)

    scale = float(out_brevitas.scale[0, 0])  

    weights_list, _ = weights_creation(ac.pi.twin, scale, ep)

    # Serialize and send the weights list; here, each weight is packed as a single byte, reflecting the int8 data type.
    packed_weights = b''.join(struct.pack('<b', weight) for weight in weights_list)
    client.send(packed_weights) # Send it all 

    print(f'Done streaming weights for episode {ep}.')

    ac.pi.train()

In [ ]:
ac = core.MLPActorCritic(device, hidden_sizes=[core.N_HIDDEN_1, core.N_HIDDEN_2]).to(device)

In [ ]:
FPGA_update(o, 0, client)

In [ ]:
ep_ret = 0
acts, obs, rews = [], [], []
ep_num = 0

if (ep_num % core.EPISODE_PER_EPOCH) == 0:
    test_flag = True
else:
    test_flag = False

print(f'Ready for episode: {ep_num}, test_flag: {test_flag}.')

# Receive obs and actions from FPGA
observations, actions = core.receive_data_episode(client, core.N_STEPS * core.N_INPUT, core.N_STEPS)


In [ ]:
observations

In [ ]:
ch1 = [obs[0] for obs in observations]

In [ ]:
acts = [act*core.ACT_LIM for act in actions]

In [ ]:
acts

In [ ]:
max(ch1)

In [ ]:
max_V = 0.18
for step in range(80):

    target = step/80*max_V
    
    print(target)